In [3]:
import pandas as pd

In [ ]:
<data>      
    <items>    
        <item name="item1">item1abc</item>    
        <item name="item2">item2abc</item>    
    </items>    
</data>  

In [ ]:
<menu>    
    <dish name="Кура">  
        <price>40</price>  
        <weight>300</weight>  
        <class>Мясо</class>  
    </dish>
    <dish name="Греча">  
        <price>20</price>  
        <weight>200</weight>  
        <class>Крупа</class>  
    </dish>
</menu> 

Файлы XML не всегда имеют жёсткую структуру и не обязаны ее иметь, но чаще всего какая-то структура внутри файла будет. Почему? Потому что обычно XML не пишут вручную. Такие файлы генерируются и читаются кодом. Поэтому при наличии понятной структуры обработка файла становится намного проще.

Как мы увидели выше, данные в формате XML имеют древовидную структуру. Что такое дерево? Это структура, которая имеет узлы и связи между ними. Самый верхнеуровневый узел называется корнем, а всё, что находится в самом низу, называется листьями. В примере выше корнем является menu, а листьями, например, price и weight. 

У menu есть дети (потомки)— это два узла dish, имеющие разный атрибут name.

In [4]:
import xml.etree.ElementTree as ET

In [5]:
tree = ET.parse('menu.xml')    
root = tree.getroot() 
# Здесь в переменной tree мы читаем всё дерево из XML файла. 
# После этого в переменную root записываем корневой узел дерева tree.

In [4]:
root.getchildren()#список детей (потомков) этого узла
# Метод возвращает список. Если у узла нет детей, то он вернет пустой список [].

/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """Entry point for launching an IPython kernel.


[<Element 'dish' at 0x7f97e688ad70>, <Element 'dish' at 0x7f97e6895050>]

In [5]:
root.getchildren()[0] 

/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """Entry point for launching an IPython kernel.


<Element 'dish' at 0x7f97e688ad70>

In [6]:
root[0]

<Element 'dish' at 0x7f97e688ad70>

У узлов могут быть параметры (атрибуты). Например, у узлов dish мы видели атрибут name. Мы можем обратиться к атрибутам объекта с помощью команды attrib. Здесь может возникнуть небольшая путаница, потому что мы говорим о двух разных определениях слова "атрибут" в нашем контексте:
1. Атрибут у тега (как name у dish).
2. Атрибут объекта (переменная класса). 

В данном случае мы берем объект типа ElementTree.Element, у которого есть атрибут attrib. В этом атрибуте объекта хранится словарь с атрибутами данного узла.

In [7]:
root[0].attrib

{'name': 'Кура'}

В узлах XML часто хранятся количественные показатели. Эти показатели хранятся в виде текста, и прочитать их можно, обратившись к атрибуту text у соответствующего объекта типа ElementTree.Element. 

In [8]:
root[0][0]

<Element 'price' at 0x7f97e688ae30>

In [9]:
root[0][0].text

'40'

Таким образом можно пройтись по всему дереву XML и прочитать значения в листьях. Если файл имеет четкую структуру, его во многих случаях можно превратить в таблицу и работать с ней уже в pandas. 

Если вы хотите прочитать название тега для конкретного узла, можно использовать атрибут tag.

In [10]:
root[0][1].tag

'weight'

Так как к узлам можно обращаться как к спискам, можно использовать циклы для итерации по детям узлов.

In [11]:
# напечатать все значения в листьях,
for elem in root:  
    for subelem in elem:  
        print(elem.attrib['name'], subelem.tag, subelem.text)  
    print() 

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



Что здесь происходит?

1. В первом цикле перебираем детей корня дерева (root). Дети появляются в переменной elem, это объекты dish.
2. Во втором цикле перебираем детей каждого блюда, то есть, каждого объекта dish. Этими детьми являются параметры блюда.
3. После этого выводим название блюда, название параметра и значение параметра.
4. Дополнительный print() в цикле верхнего уровня, чтобы было удобнее читать вывод.

In [12]:
len(root) # количество детей узла

2

In [13]:
len(root[0])

3

#### XML в pd.DataFrame

In [6]:
df_index = ['name', 'price', 'weight', 'class']  
df = pd.DataFrame(columns=df_index)  
  
for elem in root:  
    elements = [elem.get('name'), elem[0].text, elem[1].text, elem[2].text]  
    df = df.append(pd.Series(elements, index=df_index), ignore_index=True) 

1. Мы задали названия столбцов в новой таблице и создали пустой DataFrame.
2. Затем мы прошлись по всем детям из корня нашего дерева и составили строки (pd.Series), состоящие из содержимого этих элементов: взяли атрибут name у узла и значения всех его детей, которые содержат нужные нам данные. Можно заметить, что для получения названия продукта мы использовали метод get(). Это можно делать, так как атрибуты узла хранятся в виде словаря и get() — один из способов получить значения словаря, зная соответствующие ключи. 
3. После этого мы добавили новую строку в DataFrame с помощью метода append().

In [23]:
df

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


In [15]:
root[0].get('name')

'Кура'

#### XML в JSON

Для Python есть несколько сторонних библиотек, которые позволяют трансформировать XML в JSON. Одна из таких библиотек (пакетов) — xmljson. Есть несколько принятых соглашений (conventions) по превращению XML в JSON. 

In [ ]:
<p id="1">text</p>  

Тогда для соглашения badgerfish данные будут выглядеть так:

In [ ]:
# Названия атрибутов предваряются символом @, значения помечаются символом $. 
{  
  'p': {  
    '@id': 1,  
    '$': 'text'  
  }  
} 

Соглашение gdata не поддерживает атрибуты. Значение помечено как $t.

In [ ]:
{  
  'p': {  
    '$t': 'text'  
  }  
}  

Соглашение parker Атрибуты также не поддерживаются. Видно, что это самый простой способ представить XML в виде JSON: значение тега записывается как значение в словаре, где ключ — название тега.

In [ ]:
{  
  'p': 'text'  
}  

In [2]:
import xmljson  
xmljson.parker.data(root)  # можно передать название другого соглашения и будет норм xmljson.gdata.data(root)  

In [ ]:
# JSON обратно в XML
parker_json = xmljson.parker.data(root)  
back_to_xml = xmljson.parker.etree(parker_json) 

В переменной back_to_xml будет храниться наш изначальный XML-файл в формате списка, содержащего дочерние узлы корня дерева

#### Запись в XML-файл

In [7]:
# Чтобы создать корень дерева, нужно использовать метод Element() из класса ElementTree
new_root = ET.Element('menu')  

Теперь мы можем добавлять новые узлы в наше дерево, используя метод SubElement из того же класса

In [8]:
dish_1 = ET.SubElement(new_root, 'dish')  
dish_2 = ET.SubElement(new_root, 'dish')  
new_root.getchildren() 
#В метод SubElement мы передали первым аргументом узел, в который добавляем потомка. 
#Вторым аргументом мы передали название нового тега.

/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[<Element 'dish' at 0x7f9d4abf9050>, <Element 'dish' at 0x7f9d4abf9590>]

In [9]:
#Атрибуты можно добавить с помощью метода set(), передав первым параметром название атрибута, а вторым — его значение
dish_1.set('name', 'Кура')  

In [10]:
# Значение тега можно задать через уже знакомый вам параметр text
dish_1.text = 'Белок'

Сначала превратим объект типа ElementTree.Element в строку (str) c помощью метода tostring(), передав наше новое дерево как аргумент

In [11]:
new_root_string = ET.tostring(new_root)  

In [12]:
with open("new_menu.xml", "wb") as f:  
    f.write(new_root_string)

In [ ]:
# ElementTree вернул нам строку в байтовом представлении. 
# Мы создали новый файл new_menu.xml и записали в него результат.
# Почему так получилось? Мы не указали кодировку для записи, но для bytes-формата нет возможности указать 
# кодировку при записи. Что делать? Можно записать файл, используя сам класс ElementTree

In [13]:
ET.ElementTree(new_root).write('new_menu_good.xml', encoding="utf-8")